# Overall analysis of consumption and distribution of the measure 'Midlertidig lønnstilskudd' in the period 2018-2020

In [ ]:
import pandas as pandas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import dataverk_tools.themes.nav as nav
from io import StringIO
import requests
import itertools
import plotly.graph_objs as go
from matplotlib.ticker import FormatStrFormatter



In [ ]:
import dataverk_tools.themes.navdesign as theme
import plotly.io as plio
plio.templates.default = theme.plotly_template
#theme.colors.navColors


In [ ]:
from dataverk_vault import api as vault_api
import cx_Oracle
import os
import json

In [ ]:
from dataverk_vault.api import set_secrets_as_envs
set_secrets_as_envs()

# get secrets from vault

user_dvh = os.environ["DVH_USER"]
pw_dvh = os.environ['DVH_PWD']
dvh_service = os.environ['DVH_SERVICE_NAME']
dvh_port    = os.environ['DVH_PORT']
dvh_cn      = os.environ['DVH_CN']


In [ ]:
# Create string for database connection
my_dsn = cx_Oracle.makedsn( dvh_cn, dvh_port, service_name=dvh_service)

# Creates link to data warehouse
con = cx_Oracle.connect(user=user_dvh, password=pw_dvh, dsn=my_dsn)

#print("connection success")

In [ ]:
import sys
sys.path.append('~/arbeidsretta-tiltak/python')
import mappings
import importlib
#importlib.reload(mappings)

In [ ]:
# All labour market measures that are  in DWH(both active and inactive measures)
tiltak_DWH= mappings.tiltak
#tiltak_DWH

In [ ]:
# Extracting labour market measures that are active today
active_tiltak = ["AMO","AMOE","ARBFORB","ARBRRHDAG","ARBTREN","AVKLARAG","ENKELAMO","ENKFAGYRKE",
                 "GRUFAGYRKE","GRUPPEAMO","HOYEREUTD","INDJOBSTOT","INDOPPFAG",
                 "INKLUTILS","JOBBK","MENTOR","MIDLONTIL","REFINO","SUPPEMP",
                 "UTVAOONAV","VARLONTIL","VASV","VATIAROR"]
tiltak_navn = {key: tiltak_DWH[key] for key in active_tiltak}
tiltak_navn["MIDLONTIL"] = 'Midlertidig_lønnstilskudd'
#tiltak_navn

## Collection of user defined function used in this  analysis

In [ ]:
sys.path.append('~/arbeidsretta-tiltak/python/tlk_lonn')
import prep_utils
import plot_utils

## How does the gender balance look in all measures in the period 2018-2020?

In [ ]:
# common dataframe for gender balance 
kjonn_tiltak = prep_utils.fetch_sql_file("../Lønntilskudd_sql/tiltak_kjønn.sql", con)


kjonn_tiltak_navn = prep_utils.map_tiltak_navn(kjonn_tiltak)
kjonn_tiltak_kjonn_id = prep_utils.replace_kjonn_id(kjonn_tiltak)
#kjonn_tiltak_kjonn_id.head(5)



In [ ]:
# Dataframe for all measures by gender balance
tiltak_alle =kjonn_tiltak_kjonn_id.copy()
#tiltak_alle.head(5)



In [ ]:

kjonn_alle_tiltak_andel = prep_utils.find_kjonn_percentage(tiltak_alle,'COUNT')
#kjonn_alle_tiltak_andel


In [ ]:
# Data frame for temporary wage measures for Gender Balance
# Fetching rows with column name - titak navn : Midlertidig_lønnstilskudd
kjonn_mid_lønn = kjonn_tiltak_kjonn_id[(kjonn_tiltak_kjonn_id['TILTAK_NAVN'] == 'Midlertidig_lønnstilskudd') ]
#kjonn_mid_lønn.head(5)



In [ ]:
# found duplicates and droping those rows
kjonn_mid_lønn[kjonn_mid_lønn['PERSON_ID'].duplicated() == True]
kjonn_mid_lønn = kjonn_mid_lønn.drop_duplicates('PERSON_ID').sort_index()
#kjonn_mid_lønn.head(5)

In [ ]:

kjonn_mid_lønn_andel = prep_utils.find_kjonn_percentage(kjonn_mid_lønn,'COUNT')
#kjonn_mid_lønn_andel



In [ ]:
# Dataframe for work training measures for Gender Balance
# Fetching rows with column name - titak navn : Arbeidstrening
kjonn_arbtren = kjonn_tiltak_kjonn_id[(kjonn_tiltak_kjonn_id['TILTAK_NAVN'] == 'Arbeidstrening') ]
#kjonn_arbtren[kjonn_arbtren['PERSON_ID'] ==2815291]
#kjonn_arbtren

In [ ]:
# found duplicates and droping those rows
kjonn_arbtren[kjonn_arbtren['PERSON_ID'].duplicated() == True]
kjonn_arbtren = kjonn_arbtren.drop_duplicates('PERSON_ID').sort_index()
#kjonn_arbtren

In [ ]:

kjonn_arbtren_andel = prep_utils.find_kjonn_percentage(kjonn_arbtren,'COUNT')
kjonn_arbtren_andel




## How does the proportion of women and men who have completed all measures, temporary wage subsidy and work training in the period 2018-2020

In [ ]:
# preparing dataframe
# concating above 3 dataframe togather 
concat_kjonn = pandas.concat([kjonn_alle_tiltak_andel,kjonn_arbtren_andel,kjonn_mid_lønn_andel], axis=1)
#concat_kjonn
# Renaming the columns after concating dataframe                      
concat_kjonn.columns._data[1] = 'Count_Alle_tiltak'
concat_kjonn.columns._data[2] = 'Alle_tiltak'
concat_kjonn.columns._data[4] = 'Count_Arbeidstrening'
concat_kjonn.columns._data[5] = 'Arbeidstrening'
concat_kjonn.columns._data[7] = 'Count_Midlertidig_lønnstilskudd'
concat_kjonn.columns._data[8] = 'Midlertidig_lønnstilskudd'

# Removing duplicated columns(kjonn_id)                             
concat_kjonn = concat_kjonn.loc[:,~concat_kjonn.columns.duplicated()]
# Extracting necessary columns
df = concat_kjonn[['KJONN_ID','Alle_tiltak','Arbeidstrening', 'Midlertidig_lønnstilskudd']]
#df


In [ ]:
# Making tiltak navn colum som index to plot the graph

products_list = df.values.tolist()

products_list[0].remove('Menn')
products_list[1].remove('kvinner')
#print(products_list)
#print(lst)

lst = list(df)
#print(lst)

lst_per  = lst[1:]
#print(lst_per)
Menn = products_list[0]
#print(Menn)
kvinner = products_list[1]
#print(kvinner)

index = lst_per
#print(index)
df = pandas.DataFrame({'Menn': Menn,
                'kvinner':kvinner}, index=index)
#df


In [ ]:

ax = df.plot.bar(figsize=(8,5))

for p in ax.patches:
    ax.annotate((format((p.get_height()))), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=20,
                   xytext = (0, -12), 
                   textcoords = 'offset points')
ax.axhline(50)
plt.title("Proportion of women and men who have completed all measures, temporary  wage subsidy and work training in the period 2018-2020",size = 25)
plt.legend(bbox_to_anchor=(1.0,1.0))
plt.ylabel('Andel')
plt.show()





## What is the difference in the proportion of men and women who have completed measures

In [ ]:

df_diff = df.copy()

df_diff =df_diff.round(1)
#df_diff

In [ ]:
df_diff['diff'] = (df_diff['kvinner']-df_diff['Menn'])
df_diff = df_diff.T
df_diff =df_diff.round(2)
#df_diff

In [ ]:
from matplotlib import pyplot
import plotly.express as px
from matplotlib import gridspec


labels = list(df_diff.columns)  
#print(labels)

menn = df_diff.iloc[0]
menn = menn.values.tolist()
kvinner = df_diff.iloc[1]
kvinner = kvinner.values.tolist()
#print(menn)
#print(kvinner)


diff = df_diff.iloc[2]
diff = Ratio.values.tolist()

#print(diff)
x = np.arange(len(labels)) 
width = 0.35  # the width of the bars

#fig, ax = plt.subplots()
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7,7))

#ploting axes 1
rects1 = ax1.bar(x - width/2, menn, width, label='Menn')
rects2 = ax1.bar(x + width/2, kvinner, width, label='kvinner')
ax1.set_ylim(0, 70)
ax1.axhline(50)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_ylabel('Andel (%)')
#ax1.set_title('Proportion of men and women who have completed the measures')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.legend()
ax1.bar_label(rects1, padding=3)
ax1.bar_label(rects2, padding=3)

#ploting axes 2
rects3 = plt.stem(x - width/2, Ratio,markerfmt = 'bo', linefmt='-',label='bias in women' )
#rects3 = ax2.bar(x + width/2, Ratio, width, label='kvinner', color ='tab:orange' )
ax2.set_ylim(-40, 20)
ax2.axhline(0)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax2.set_title('skjevhet i andel kvinner   ')
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.legend()
#ax2.bar_label(rects3, padding=3)
plt.legend()
ax2.set_ylabel('Andel (%)')

#Fill area when Ratio > 0 with green og red when Ratio < 0  
ax2.axhspan(0, 20, facecolor='green', alpha=0.5,label="Postiv utvikling")
ax2.axhspan(0, -45, facecolor='red', alpha=0.5,label="Negativ utvikling")
for x,y in zip(x, Ratio):
    plt.annotate('{}'.format(y), xy=(x,y), xytext=(0,-7), textcoords="offset points",
                    ha='right', va='bottom')


fig.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1.03))



## How was the distribution of gender in all measures by county in the period 2018 - 2020

In [ ]:
flyker_2020 = {   
                  1.0 : 'Viken', 
                  2.0 : 'Viken', 
                  6.0 : 'Viken',
                  30.0: 'Viken',
                  3.0 : 'Oslo',
                  4.0 : 'Innlandet', 
                  5.0 : 'Innlandet', 
                  34.0: 'Innlandet',
                  7.0 : 'Vestfold og Telemark',
                  8.0 : 'Vestfold og Telemark',
                  38.0: 'Vestfold og Telemark',
                  9.0 : 'Agder',
                  10.0: 'Agder',
                  42.0: 'Agder',
                  11.0: 'Rogaland',
                  12.0: 'Vestland',
                  14.0: 'Vestland',
                  46.0: 'Vestland',
                  15.0: 'Møre og Romsdal',
                  18.0: 'Nordland',
                  19.0: 'Troms og Finnmark',
                  20.0: 'Troms og Finnmark',
                  54.0: 'Troms og Finnmark',
                  21.0: 'Svalbard',
                  50.0: 'Trøndelag',
                  16.0: 'Trøndelag',
                  17.0: 'Trøndelag',
                  0   : 'Annet bosted',
                  -1.0: 'Annet bosted',
    
                 
              }

In [ ]:
# common data framework for gender balance by county
flyke_tiltak = prep_utils.fetch_sql_file("../Lønntilskudd_sql/tiltak_flyke.sql", con)
flyke_tiltak_navn = prep_utils.map_tiltak_navn(flyke_tiltak)
flyke_tiltak_kjonn_id = prep_utils.replace_kjonn_id(flyke_tiltak)
flyke_tiltak_flyke_navn = prep_utils.fetch_flykenummer_map_flyke_navn(flyke_tiltak)
#flyke_tiltak_flyke_navn



In [ ]:
# dataframe for all measures on gender balance by county
flyke_alle_tiltak = flyke_tiltak_flyke_navn.copy()



In [ ]:
# dataframe for for temporary wage subsidy on gender balance by county

flyke_mid_lønn =flyke_tiltak_flyke_navn.copy()
flyke_mid_lønn = flyke_mid_lønn[flyke_mid_lønn.TILTAK_NAVN=='Midlertidig_lønnstilskudd']
#flyke_mid_lønn


In [ ]:
# dataframe for work training measures on gender balance by county
flyke_arbtren =flyke_tiltak_flyke_navn.copy()
flyke_arbtren = flyke_arbtren[flyke_arbtren.TILTAK_NAVN=='Arbeidstrening']
#flyke_arbtren



In [ ]:
flyke_alle_tiltak_andel = flyke_percentage_alletiltak(flyke_alle_tiltak,'flyke_new','COUNT')
flyke_alle_tiltak_andel



In [ ]:
andel_flyke_mid_lønn = flyke_percentage_arbtren_midlønn(flyke_mid_lønn,'flyke_new','PERSON_ID')


In [ ]:
andel_flyke_arbtren = flyke_percentage_arbtren_midlønn(flyke_arbtren,'flyke_new','PERSON_ID')
#andel_flyke_arbtren

## How does the proportion of women and men who have completed all measures, temporary wage subsidy and work training in the period 2018-2020 by fcounties and by age group

In [ ]:
# preparing dataframe by by  concating above 3 dataframe togather to make sub plot by counties 
concat_kjonn_flyke = pandas.concat([flyke_alle_tiltak_andel,andel_flyke_arbtren,andel_flyke_mid_lønn], axis=1)
#concat_kjonn_flyke



In [ ]:
# preparing common dataframe for sub plot by age groups

aldergrupp = kjonn_tiltak_kjonn_id.copy()
aldergrupp = aldergrupp.drop(aldergrupp[aldergrupp['AGE'] == 17].index)

#aldergrupp

In [ ]:
# Addind new column(alder_gruppen)by binning the age 

num_of_bins = [18, 30, 40, 50, float('inf')]

labels=['18-30', '30-40', '40-50', '50+']

aldergrupp['alder_gruppen']=pandas.cut(aldergrupp['AGE'], bins=num_of_bins, labels=labels,include_lowest= True)


aldergrupp


In [ ]:
# Dataframe for alle measures by age group
alle_tiltak_alder = aldergrupp.copy()
#alle_tiltak_alder

In [ ]:
# Dataframe for temporary wages by age group
alder_mid_lønn = aldergrupp[aldergrupp.TILTAK_NAVN=='Midlertidig_lønnstilskudd']
#alder_mid_lønn

In [ ]:
# Dataframe for work traing measure age group

alder_arbtren = aldergrupp[aldergrupp.TILTAK_NAVN=='Arbeidstrening']
#alder_arbtren


In [ ]:
alder_alle_tiltak_andel = flyke_percentage_alletiltak(alle_tiltak_alder,'alder_gruppen','COUNT')
#alder_alle_tiltak_andel

In [ ]:
andel_alder_mid_lønn = flyke_percentage_arbtren_midlønn(alder_mid_lønn,'alder_gruppen','PERSON_ID')
#andel_alder_mid_lønn

In [ ]:
andel_alder_arbren = flyke_percentage_arbtren_midlønn(alder_arbtren,'alder_gruppen','PERSON_ID')
#andel_alder_arbren

In [ ]:
# preparing dataframe by concating above 3 dataframe togather to make sub plot by age group 
concat_kjonn_alder = pandas.concat([alder_alle_tiltak_andel,andel_alder_arbren,andel_alder_mid_lønn], axis=1)
#concat_kjonn_alder


In [ ]:
plot_utils.create_sub_plot(concat_kjonn_flyke,'flyke_new')


In [ ]:
plot_utils.create_sub_plot(concat_kjonn_alder,'alder_gruppen')



## What measures users have had before the temporary wage subsidy in the period 2018-2020

In [ ]:
# Dataframe for brukerene(men and women) in temporary wage 
df_sankey = prep_utils.fetch_sql_file("../Lønntilskudd_sql/tiltak_før_Midlertidiglønnstilskudd.sql", con)

df_sankey_tiltak_navn = prep_utils.map_tiltak_navn(df_sankey)
df_sankey_kjonn_id = prep_utils.replace_kjonn_id(df_sankey)



In [ ]:
# Dataframe for men in temporary wage 

Menn_sankey = df_sankey[df_sankey['KJONN_ID'] != 'kvinner']

#Menn_sankey

In [ ]:
# Dataframe for women in temporary wage 

kvinner_sankey = df_sankey.loc[df_sankey['KJONN_ID'] != 'Menn']
#kvinner_sankey

In [ ]:
df_sankey = df_sankey.groupby('PERSON_ID')
kvinner_sankey = kvinner_sankey.groupby('PERSON_ID')
Menn_sankey = Menn_sankey.groupby('PERSON_ID')



In [ ]:

my_dict_overall = plot_utils.create_sankey_midlønn(df_sankey)
#print(my_dict_overall)



In [ ]:
my_dict_kvinner = plot_utils.create_sankey_midlønn(kvinner_sankey)

#print(my_dict_kvinner)

In [ ]:
my_dict_man = plot_utils.create_sankey_midlønn(Menn_sankey)
#print(my_dict_man)

In [ ]:
plot_utils.draw_sankey_midlønn(my_dict_overall,my_dict_man,my_dict_kvinner)

# Proportion shows the transition of users from work training to various measures in the period 2018-2020

In [ ]:
# Datafame for brukrene in work training 
arbtren_brukrne = df_sankey


In [ ]:
# Datafame for men in work training

arbtren_menn = Menn_sankey

In [ ]:
# Datafame for women in work  training


arbtren_kvinner = kvinner_sankey



In [ ]:


plot_arbtren_overall = plot_utils.create_sankey_arbtren(arbtren_brukrne)
#print(arbtren_overall)





In [ ]:
plot_arbtren_kvinner = plot_utils.create_sankey_arbtren(arbtren_kvinner)

#print(arbtren_kvinner)

In [ ]:
plot_arbtren_man = plot_utils.create_sankey_arbtren(arbtren_menn)

#print(arbtren_man)

In [ ]:
plot_utils.draw_sankey_arbtren(plot_arbtren_overall,plot_arbtren_man,plot_arbtren_kvinner)